In [1]:
print('introduction to torchscript.')

introduction to torchscript.


In [2]:
import torch
print(torch.__version__)

1.2.0


In [3]:
print('basics of pytorch model authoring')
class MyCell(torch.nn.Module):
    def __init(self):
        super(MyCell, self).__init__()
        
    def forward(self, x, h):
        new_h = torch.tanh(x + h)
        return new_h, new_h

my_cell = MyCell()
x = torch.rand(3, 4)
h = torch.rand(3, 4)
print(my_cell(x, h))


basics of pytorch model authoring
(tensor([[0.8584, 0.8193, 0.8820, 0.9504],
        [0.7409, 0.8393, 0.7489, 0.7216],
        [0.6857, 0.4367, 0.0745, 0.7348]]), tensor([[0.8584, 0.8193, 0.8820, 0.9504],
        [0.7409, 0.8393, 0.7489, 0.7216],
        [0.6857, 0.4367, 0.0745, 0.7348]]))


In [9]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)
    
    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))


MyCell(
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[ 0.1761,  0.6880,  0.6352,  0.7515],
        [-0.1975,  0.9413,  0.8519,  0.5038],
        [-0.1288,  0.7946,  0.7693,  0.8332]], grad_fn=<TanhBackward>), tensor([[ 0.1761,  0.6880,  0.6352,  0.7515],
        [-0.1975,  0.9413,  0.8519,  0.5038],
        [-0.1288,  0.7946,  0.7693,  0.8332]], grad_fn=<TanhBackward>))


In [10]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.dg = MyDecisionGate()
        self.linear = torch.nn.Linear(4, 4)
        
    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h
    
my_cell = MyCell()
print(my_cell)
print(my_cell(x, h))


MyCell(
  (dg): MyDecisionGate()
  (linear): Linear(in_features=4, out_features=4, bias=True)
)
(tensor([[ 0.8152, -0.2037,  0.4657,  0.6881],
        [ 0.4889,  0.5876,  0.8306,  0.2261],
        [ 0.7118, -0.2396,  0.6319,  0.7876]], grad_fn=<TanhBackward>), tensor([[ 0.8152, -0.2037,  0.4657,  0.6881],
        [ 0.4889,  0.5876,  0.8306,  0.2261],
        [ 0.7118, -0.2396,  0.6319,  0.7876]], grad_fn=<TanhBackward>))


In [13]:
print('tracing...')
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)
    
    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h, new_h

my_cell = MyCell()
x, h = torch.rand(3,4), torch.rand(3,4)
traced_cell = torch.jit.trace(my_cell, (x, h))
print(traced_cell)
print(traced_cell(x, h))
print(traced_cell.graph)
print(traced_cell.code)

tracing...
TracedModule[MyCell](
  (linear): TracedModule[Linear]()
)
(tensor([[ 0.8107,  0.6864, -0.0111, -0.0114],
        [ 0.8867,  0.5783, -0.3450,  0.6337],
        [ 0.9471,  0.4010, -0.1775,  0.4589]],
       grad_fn=<DifferentiableGraphBackward>), tensor([[ 0.8107,  0.6864, -0.0111, -0.0114],
        [ 0.8867,  0.5783, -0.3450,  0.6337],
        [ 0.9471,  0.4010, -0.1775,  0.4589]],
       grad_fn=<DifferentiableGraphBackward>))
graph(%self : ClassType<MyCell>,
      %input : Float(3, 4),
      %h : Float(3, 4)):
  %1 : ClassType<Linear> = prim::GetAttr[name="linear"](%self)
  %weight : Tensor = prim::GetAttr[name="weight"](%1)
  %bias : Tensor = prim::GetAttr[name="bias"](%1)
  %6 : Float(4!, 4!) = aten::t(%weight), scope: MyCell/Linear[linear] # /home/rong/PycharmProjects/PyTorchTutorals/venv/lib/python3.6/site-packages/torch/nn/functional.py:1369:0
  %7 : int = prim::Constant[value=1](), scope: MyCell/Linear[linear] # /home/rong/PycharmProjects/PyTorchTutorals/venv/lib/pyt

In [17]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

class MyCell(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)
        
    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h, new_h
    
my_cell = MyCell(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))
# print(my_cell)
# print(my_cell(x, h))
print(traced_cell.code)

def forward(self,
    input: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = self.linear
  weight = _0.weight
  bias = _0.bias
  x = torch.addmm(bias, input, torch.t(weight), beta=1, alpha=1)
  _1 = torch.tanh(torch.add(x, h, alpha=1))
  return (_1, _1)



/home/rong/PycharmProjects/PyTorchTutorals/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
scripted_gate = torch.jit.script(MyDecisionGate())
my_cell = MyCell(scripted_gate)
traced_cell = torch.jit.script(my_cell)
print(traced_cell.code)
x, h = torch.rand(3, 4), torch.rand(3, 4)
traced_cell(x, h)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tuple[Tensor, Tensor]:
  _0 = self.linear
  _1 = _0.weight
  _2 = _0.bias
  if torch.eq(torch.dim(x), 2):
    _3 = torch.__isnot__(_2, None)
  else:
    _3 = False
  if _3:
    bias = ops.prim.unchecked_unwrap_optional(_2)
    ret = torch.addmm(bias, x, torch.t(_1), beta=1, alpha=1)
  else:
    output = torch.matmul(x, torch.t(_1))
    if torch.__isnot__(_2, None):
      bias0 = ops.prim.unchecked_unwrap_optional(_2)
      output0 = torch.add_(output, bias0, alpha=1)
    else:
      output0 = output
    ret = output0
  _4 = torch.gt(torch.sum(ret, dtype=None), 0)
  if bool(_4):
    _5 = ret
  else:
    _5 = torch.neg(ret)
  new_h = torch.tanh(torch.add(_5, h, alpha=1))
  return (new_h, new_h)



(tensor([[ 0.9194,  0.8591,  0.2544,  0.0303],
         [ 0.6037,  0.6210,  0.3884,  0.0590],
         [ 0.7844,  0.8996,  0.5261, -0.2167]],
        grad_fn=<DifferentiableGraphBackward>),
 tensor([[ 0.9194,  0.8591,  0.2544,  0.0303],
         [ 0.6037,  0.6210,  0.3884,  0.0590],
         [ 0.7844,  0.8996,  0.5261, -0.2167]],
        grad_fn=<DifferentiableGraphBackward>))

In [20]:
print('Mixing scripting and tracing....')
class MyRNNLoop(torch.nn.Module):
    def __init__(self):
        super(MyRNNLoop, self).__init__()
        self.cell = torch.jit.trace(MyCell(scripted_gate), (x, h))
    
    def forward(self, xs):
        h, y = torch.zeros(3, 4), torch.zeros(3, 4)
        for i in range(xs.size(0)):
            y, h = self.cell(xs[i], h)
        return y, h
    
rnn_loop = torch.jit.script(MyRNNLoop())
print(rnn_loop.code)

Mixing scripting and tracing....
def forward(self,
    xs: Tensor) -> Tuple[Tensor, Tensor]:
  h = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  y = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  y0, h0 = y, h
  for i in range(torch.size(xs, 0)):
    _0 = self.cell
    _1 = torch.select(xs, 0, i)
    _2 = _0.linear
    weight = _2.weight
    bias = _2.bias
    _3 = torch.addmm(bias, _1, torch.t(weight), beta=1, alpha=1)
    _4 = torch.gt(torch.sum(_3, dtype=None), 0)
    if bool(_4):
      _5 = _3
    else:
      _5 = torch.neg(_3)
    _6 = torch.tanh(torch.add(_5, h0, alpha=1))
    y0, h0 = _6, _6
  return (y0, h0)



In [21]:
class WrapRNN(torch.nn.Module):
    def __init__(self):
        super(WrapRNN, self).__init__()
        self.loop = torch.jit.script(MyRNNLoop())
        
    def forward(self, xs):
        y, h = self.loop(xs)
        return torch.relu(y)
    
traced = torch.jit.trace(WrapRNN(), (torch.rand(10, 3, 4)))
print(traced.code)

def forward(self,
    argument_1: Tensor) -> Tensor:
  _0 = self.loop
  h = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  h0 = h
  for i in range(torch.size(argument_1, 0)):
    _1 = _0.cell
    _2 = torch.select(argument_1, 0, i)
    _3 = _1.linear
    weight = _3.weight
    bias = _3.bias
    _4 = torch.addmm(bias, _2, torch.t(weight), beta=1, alpha=1)
    _5 = torch.gt(torch.sum(_4, dtype=None), 0)
    if bool(_5):
      _6 = _4
    else:
      _6 = torch.neg(_4)
    h0 = torch.tanh(torch.add(_6, h0, alpha=1))
  return torch.relu(h0)



In [22]:
print('Saving and Loading models')
traced.save('wrapped_rnn.zip')


Saving and Loading models


In [23]:
loaded = torch.jit.load('wrapped_rnn.zip')
print(loaded)
print(loaded.code)


ScriptModule(
  (loop): ScriptModule(
    (cell): ScriptModule(
      (dg): ScriptModule()
      (linear): ScriptModule()
    )
  )
)
def forward(self,
    argument_1: Tensor) -> Tensor:
  _0 = self.loop
  h = torch.zeros([3, 4], dtype=None, layout=None, device=None, pin_memory=None)
  h0 = h
  for i in range(torch.size(argument_1, 0)):
    _1 = _0.cell
    _2 = torch.select(argument_1, 0, i)
    _3 = _1.linear
    weight = _3.weight
    bias = _3.bias
    _4 = torch.addmm(bias, _2, torch.t(weight), beta=1, alpha=1)
    _5 = torch.gt(torch.sum(_4, dtype=None), 0)
    if bool(_5):
      _6 = _4
    else:
      _6 = torch.neg(_4)
    h0 = torch.tanh(torch.add(_6, h0, alpha=1))
  return torch.relu(h0)

